In [1]:
import pandas as pd
import utils
import numpy as np
import scipy.sparse as sps


In [2]:
tracks = pd.read_csv('../input/tracks.csv')
train = pd.read_csv('../input/train.csv')
target = pd.read_csv('../input/target_playlists.csv')

In [3]:
icm_csr = utils.build_icm_csr(tracks)
urm_csr = utils.build_urm_csr(train)

/home/antonio/anaconda3/lib/python3.6/site-packages/scipy/sparse/compressed.py:746: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [4]:

from Notebooks_utils.data_splitter import train_test_holdout

URM_train, URM_test = train_test_holdout(urm_csr, train_perc = 0.8)
URM_train, URM_validation = train_test_holdout(URM_train, train_perc = 0.9)

In [5]:
 URM_train


<50446x20635 sparse matrix of type '<class 'numpy.float32'>'
	with 873084 stored elements in Compressed Sparse Row format>

In [6]:
URM_test

<50446x20635 sparse matrix of type '<class 'numpy.float32'>'
	with 242140 stored elements in Compressed Sparse Row format>

In [7]:
URM_validation

<50446x20635 sparse matrix of type '<class 'numpy.float32'>'
	with 96567 stored elements in Compressed Sparse Row format>